In [ ]:
OPEN_AI_SECRET_KEY="example"

NEO4J_URI="neo4j+s://example.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="example"
AURA_INSTANCEID="example"
AURA_INSTANCENAME="Instance0"

NEO4J_DB = "neo4j"

Importing turtle file into AuraDB

In [ ]:
!pip -q install rdflib rdflib-neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 6.1 MB/s eta 0:00:00


In [ ]:
# importing ttl file
from google.colab import files
uploaded = files.upload()
ttl_path = list(uploaded.keys())[0]

Saving yang_rule_no_mandatory_on_key.ttl to yang_rule_no_mandatory_on_key.ttl


In [ ]:
# Ingest Turtle into Aura (or any Neo4j) with RDFLib-Neo4j
from rdflib import Graph
from rdflib_neo4j import Neo4jStore, Neo4jStoreConfig, HANDLE_VOCAB_URI_STRATEGY

config = Neo4jStoreConfig(
    auth_data={"uri": NEO4J_URI, "database": NEO4J_DB, "user": NEO4J_USERNAME, "pwd": NEO4J_PASSWORD},
    handle_vocab_uri_strategy=HANDLE_VOCAB_URI_STRATEGY.IGNORE,  # or SHORTEN/MAP/KEEP
    batching=True,  # optional: buffer writes for speed; remember to close()
)

g = Graph(store=Neo4jStore(config=config))

# Auto-create the required uniqueness constraint if missing:
g.open(configuration=None, create=True)  # creates CONSTRAINT on :Resource(uri)

# Parse from local file:
g.parse(ttl_path, format="turtle")       # or format="ttl"

# Or parse directly from a URL instead:
# g.parse("https://example.com/your.ttl", format="turtle")

print("Triples loaded into Neo4j via RDFLib:", len(g))
g.close(True)  # commit pending buffered writes


Uniqueness constraint on :Resource(uri) found. 
                
                
The store is now: Open
Uniqueness constraint on :Resource(uri) found. 
                
                
The store is now: Open
Triples loaded into Neo4j via RDFLib: 0
The store is now: Closed
IMPORTED 25 TRIPLES


Adding usage of LangChain

In [ ]:
# Install modern LangChain split packages
!pip -q install langchain langchain-community langchain-openai langchain-neo4j neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 11.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.12.0 requires tenacity<9.0.0,>=8.0.0, but you have tenacity 9.1.2 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPEN_AI_SECRET_KEY

In [ ]:
EMBED_MODEL = "text-embedding-3-large"   # or "text-embedding-3-small"
CHAT_MODEL  = "gpt-5"              # or "gpt-4o"

In [ ]:
# Sanity check
from neo4j import GraphDatabase

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
with driver.session() as s:
    result = s.run("RETURN 1 AS ok").single()
print("Neo4j OK:", result["ok"] == 1)

Neo4j OK: True


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jVector

emb = OpenAIEmbeddings(model=EMBED_MODEL)

# configure your label + text properties here
DOC_NODE_LABEL = "Document"                   # <-- change to your label
TEXT_PROPS     = ["title", "content"]         # <-- change to your text fields
INDEX_NAME     = "docs_embedding"             # Neo4j vector index name
EMB_PROP       = "embedding"                  # property to store vectors
KEYWORD_INDEX  = "docs_keyword"               # optional: for hybrid search

vstore = Neo4jVector.from_existing_graph(
    embedding=emb,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    node_label=DOC_NODE_LABEL,
    text_node_properties=TEXT_PROPS,
    embedding_node_property=EMB_PROP,
    index_name=INDEX_NAME,
    keyword_index_name=KEYWORD_INDEX,   # keep if you want hybrid search
    search_type="hybrid"                # or "vector"
)

print("Vector store ready (existing graph).")

Vector store ready (existing graph).


In [ ]:
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jVector

def load_graph_chunks(limit=8000):
    """
    Robust chunk builder: avoids missing property warnings
    and guarantees non-null strings for page_content.
    Adjust the property candidates to match your schema.
    """
    q = """
    MATCH (a)-[r]->(b)
    WITH
      coalesce(
        a.name, a.id, a.title, a.label, a.shortName, a.code, elementId(a)
      ) AS left_any,
      head(labels(a)) AS left_label,
      type(r) AS rel,
      coalesce(
        r.name, r.id, r.protocol, r.role, ''   // add r.interface here if you *do* have it
      ) AS r_any,
      coalesce(
        b.name, b.id, b.title, b.label, b.shortName, b.code, elementId(b)
      ) AS right_any,
      head(labels(b)) AS right_label
    WITH
      toString(left_any)  AS left,
      coalesce(left_label,'?')   AS left_label,
      rel,
      toString(r_any)     AS rname,
      toString(right_any) AS right,
      coalesce(right_label,'?')  AS right_label
    RETURN
      'Left: ' + left + ' (' + left_label + ')' +
      '\nRel: ' + rel +
      CASE WHEN rname IS NOT NULL AND rname <> '' THEN ' [' + rname + ']' ELSE '' END +
      '\nRight: ' + right + ' (' + right_label + ')' AS text,
      {left:left, right:right, rel:rel, rname:rname, left_label:left_label, right_label:right_label} AS meta
    LIMIT $limit
    """
    with driver.session() as s:
        rows = s.run(q, limit=limit).data()

    # Defensive: keep only rows that produced text
    docs = [Document(page_content=r.get("text", ""), metadata=r.get("meta", {}))
            for r in rows if r.get("text")]
    return docs

docs = load_graph_chunks(limit=8000)
print(f"Built {len(docs)} chunk(s).")
if docs:
    print("Example:\n", docs[0].page_content[:300])

emb = OpenAIEmbeddings(model=EMBED_MODEL)

vstore = Neo4jVector.from_documents(
    documents=docs,
    embedding=emb,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    # customize storage / indexes:
    node_label="Chunk",
    text_node_property="text",        # <-- singular
    embedding_node_property="embedding",
    index_name="chunk_embedding",
    search_type="hybrid",             # creates vector + keyword indexes
    keyword_index_name="chunk_keyword"
)

print("Vector store ready (chunks from triples).")

Built 3524 chunk(s).
Example:
 Left: 4:8606e1fe-fb64-473a-89ee-50c8506f5ac1:1 (Resource)
Rel: domain
Right: 4:8606e1fe-fb64-473a-89ee-50c8506f5ac1:986 (Resource)
Vector store ready (chunks from triples).


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model=CHAT_MODEL, temperature=0)

# hybrid works well for acronyms (AMF/SMF/N2/N11/5QI); fallback to "similarity" if needed
try:
    retriever = vstore.as_retriever(search_type="hybrid", search_kwargs={"k": 6})
except Exception:
    retriever = vstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

TELECOM_PROMPT = PromptTemplate.from_template(
    """You are a 5G/Open5GS expert. Your purpose is to translate high-level requirements
    for a 5GCore network into a YANG configuration. Analyze carefully all of the requirements
    for a YANG format and keep it strict. Ensure that YANG format follows all the rules to be
    validated successfully by libyang tool.

Question:
{question}

Context:
{context}

At the end, list bullet 'evidence lines' in the format (Left|Rel|Right)."""
)

# TELECOM_PROMPT = PromptTemplate.from_template(
#     """You are a 5G/Open5GS expert. Use standards-aware, precise language.
# Expand acronyms on first use (e.g., AMF—Access and Mobility Management Function).
# If interfaces appear, mention their purpose (e.g., N11—AMF↔SMF, session control).

# Question:
# {question}

# Context:
# {context}

# Answer clearly. At the end, list 2–5 bullet 'evidence lines' in the format (Left|Rel|Right)."""
# )

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": TELECOM_PROMPT},
    return_source_documents=True
)

query = "Enable each network function to declare the services it exposes, including version and supported protocols."
resp = qa({"query": query})

print("Q:", query, "\n")
print(resp["result"])
print("\nSources:")
for d in resp["source_documents"][:5]:
    print("-", (d.page_content.splitlines() or [d.page_content[:120]])[0])


/tmp/ipython-input-922585009.py:51: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resp = qa({"query": query})


Q: Enable each network function to declare the services it exposes, including version and supported protocols. 

module open5gs-nf-services {
  yang-version 1.1;
  namespace "urn:open5gs:params:xml:ns:yang:open5gs-nf-services";
  prefix og-nfs;

  organization "Open5GS Community";
  contact
    "https://open5gs.org/";
  description
    "This module allows each 5GC network function (NF) to declare the services
     it exposes, including the API version and supported protocols.";

  revision 2025-09-03 {
    description
      "Initial revision.";
  }

  typedef protocol-type {
    type enumeration {
      enum http2 {
        description "HTTP/2 (RFC 7540), typical for 3GPP SBI over TLS.";
      }
      enum http1-1 {
        description "HTTP/1.1 (RFC 9112).";
      }
      enum http3 {
        description "HTTP/3 (RFC 9114) over QUIC.";
      }
    }
    description
      "Finite set of supported application protocols for the exposed services.";
  }

  container network-functions {
   

Flow for comparing doc loading and knowledge graph code

In [ ]:
!pip install -qU langchain langchain-openai langchain-community faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os, time, glob, textwrap
from dataclasses import dataclass
from typing import List, Dict, Any

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Reuse existing settings if you have them, else set defaults
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
assert OPENAI_API_KEY, "Set OPENAI_API_KEY first."

EMBED_MODEL = os.getenv("EMBED_MODEL", "text-embedding-3-small")  # good default for speed/cost
CHAT_MODEL  = os.getenv("CHAT_MODEL",  "gpt-4o-mini")

# If you already created vstore/qa for Neo4j earlier, we’ll reuse them.

In [ ]:
PDF_DIR = "./pdfs"     # <-- point to your folder of PDFs
PDF_GLOB = "*.pdf"

# 3a) Load PDFs
pdf_files = sorted(glob.glob(os.path.join(PDF_DIR, PDF_GLOB)))
assert pdf_files, f"No PDFs found under {PDF_DIR}. Put some files there or change PDF_DIR."

raw_docs = []
for f in pdf_files:
    try:
        loader = PyPDFLoader(f)
        raw_docs.extend(loader.load())
    except Exception as e:
        print(f"Warning: failed to load {f}: {e}")

print(f"Loaded {len(raw_docs)} pages from {len(pdf_files)} PDF(s).")

# 3b) Chunking
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
pdf_docs = splitter.split_documents(raw_docs)
# add simple, consistent metadata for debugging
for d in pdf_docs:
    d.metadata["source_file"] = d.metadata.get("source") or d.metadata.get("file_path")
    d.metadata["page"] = d.metadata.get("page", d.metadata.get("page_number"))

print(f"Chunks: {len(pdf_docs)}")

# 3c) Embeddings + FAISS vector store
emb_pdf = OpenAIEmbeddings(model=EMBED_MODEL)
pdf_vstore = FAISS.from_documents(pdf_docs, emb_pdf)

# Retriever for PDF
pdf_retriever = pdf_vstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
print("PDF vector store ready.")

Loaded 184 pages from 11 PDF(s).
Chunks: 685
PDF vector store ready.


In [ ]:
# Neo4j retriever/qa: try to reuse if already defined; else raise a helpful error
try:
    retriever_neo4j = vstore.as_retriever(
        search_type="similarity",                 # <- allowed by VectorStoreRetriever
        search_kwargs={"k": 6, "search_type": "hybrid"}  # <- passed through to Neo4jVector
    )
except Exception:
    # fallback to pure vector if hybrid isn't available (e.g., no keyword index)
    retriever_neo4j = vstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

llm = ChatOpenAI(model=CHAT_MODEL, temperature=0)

TELECOM_PROMPT = PromptTemplate.from_template(
    """You are a 5G/Open5GS expert. Your purpose is to translate high-level requirements
    for a 5GCore network into a YANG configuration. Analyze carefully all of the requirements
    for a YANG format and keep it strict. Ensure that YANG format follows all the rules to be
    validated successfully by libyang tool.

Question:
{question}

Context:
{context}

At the end, list bullet 'evidence lines' in the format (Left|Rel|Right)."""
)

qa_pdf = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=pdf_retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": TELECOM_PROMPT},
    return_source_documents=True,
)

qa_neo4j = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever_neo4j,
    chain_type="stuff",
    chain_type_kwargs={"prompt": TELECOM_PROMPT},
    return_source_documents=True,
)

print("RAG chains ready (Neo4j + PDF).")

RAG chains ready (Neo4j + PDF).


In [ ]:
@dataclass
class RunResult:
    pipeline: str
    answer: str
    time_s: float
    sources: List[Dict[str, Any]]

def _summarize_sources(sources: List[Any], kind: str) -> List[str]:
    out = []
    if kind == "pdf":
        for d in sources[:6]:
            src = d.metadata.get("source_file", "unknown.pdf")
            page = d.metadata.get("page", "?")
            first = (d.page_content or "").strip().replace("\n", " ")
            out.append(f"{os.path.basename(src)}:p{page} — {first[:160]}{'…' if len(first)>160 else ''}")
    else:  # neo4j
        for d in sources[:6]:
            first = (d.page_content or "").strip().splitlines()
            first_line = first[0] if first else ""
            out.append(first_line[:180] + ("…" if len(first_line) > 180 else ""))
    return out

def ask_both(question: str, judge: bool = True) -> Dict[str, Any]:
    # Neo4j
    t0 = time.time()
    neo = qa_neo4j({"query": question})
    neo_t = time.time() - t0

    # PDF
    t1 = time.time()
    pdf = qa_pdf({"query": question})
    pdf_t = time.time() - t1

    neo_sources = _summarize_sources(neo["source_documents"], "neo4j")
    pdf_sources = _summarize_sources(pdf["source_documents"], "pdf")

    result = {
        "neo4j": RunResult("neo4j", neo["result"], neo_t, neo_sources),
        "pdf":   RunResult("pdf",   pdf["result"], pdf_t, pdf_sources)
    }

    if not judge:
        return result

    # LLM-as-judge: which is more correct/grounded *given the retrieved contexts*?
    judge_prompt = PromptTemplate.from_template(
        """You are grading two answers to the same question using only the provided contexts.
Score on: factual correctness, grounding to context, telecom clarity, YANG format correctness.

For both you are a 5G/Open5GS expert. Your purpose is to translate high-level requirements
for a 5GCore network into a YANG configuration. Analyze carefully all of the requirements
for a YANG format and keep it strict. Ensure that YANG format follows all the rules to be
validated successfully by libyang tool. Answer should include a YANG template and a short
description of the result.

Question:
{question}

Answer A (Neo4j):
{ans_a}

Context A (Neo4j top sources):
{ctx_a}

Answer B (PDF):
{ans_b}

Context B (PDF top sources):
{ctx_b}

Respond as JSON with fields:
- "winner": one of ["neo4j","pdf","tie"]
- "rationale": 1-3 sentences explaining the decision.
"""
    )
    judge_llm = ChatOpenAI(model=CHAT_MODEL, temperature=0)
    judge_in = judge_prompt.format(
        question=question,
        ans_a=result["neo4j"].answer,
        ctx_a="\n".join(result["neo4j"].sources),
        ans_b=result["pdf"].answer,
        ctx_b="\n".join(result["pdf"].sources),
    )
    j = judge_llm.invoke(judge_in).content
    result["judge_raw"] = j
    return result

In [ ]:
import json

def show_comparison(question: str, judge: bool = True):
    res = ask_both(question, judge=judge)

    neo = res["neo4j"]
    pdf = res["pdf"]

    print("="*90)
    print("QUESTION:")
    print(question)
    print("="*90)
    print("[Neo4j] time: %.2fs" % neo.time_s)
    print(textwrap.fill(neo.answer, width=100))
    print("\nSources (Neo4j):")
    for s in neo.sources:
        print(" •", s)

    print("\n" + "-"*90 + "\n")

    print("[PDF]   time: %.2fs" % pdf.time_s)
    print(textwrap.fill(pdf.answer, width=100))
    print("\nSources (PDF):")
    for s in pdf.sources:
        print(" •", s)

    if judge and "judge_raw" in res:
        print("\n" + "="*90)
        print("LLM JUDGE:")
        try:
            j = json.loads(res["judge_raw"])
        except Exception:
            print(res["judge_raw"])
        else:
            print("Winner:", j.get("winner"))
            print("Rationale:", j.get("rationale"))

# Example:
# show_comparison("Which network function coordinates UE registration and which interfaces are involved?")

In [ ]:
import numpy as np

def max_grounding_similarity(answer: str, retrieved_docs: List[Any], emb: OpenAIEmbeddings) -> float:
    ans_vec = np.array(emb.embed_query(answer), dtype=float)
    ctx_vecs = np.array([emb.embed_query(d.page_content[:1000]) for d in retrieved_docs], dtype=float)
    # cosine similarity
    sims = (ctx_vecs @ ans_vec) / (np.linalg.norm(ctx_vecs, axis=1) * np.linalg.norm(ans_vec) + 1e-9)
    return float(np.max(sims)) if len(sims) else 0.0

def compare_with_scores(question: str):
    res = ask_both(question, judge=False)
    neo = res["neo4j"]; pdf = res["pdf"]

    # reuse the same embedder model used for PDF; it’s fine for a relative score
    emb_eval = emb_pdf

    # We need the full docs, not the summarized strings → rerun each quickly to capture docs
    neo_full = qa_neo4j({"query": question})
    pdf_full = qa_pdf({"query": question})

    neo_score = max_grounding_similarity(neo_full["result"], neo_full["source_documents"], emb_eval)
    pdf_score = max_grounding_similarity(pdf_full["result"], pdf_full["source_documents"], emb_eval)

    print(f"Grounding similarity (cosine to retrieved context):")
    print(f"  Neo4j: {neo_score:.3f}")
    print(f"  PDF  : {pdf_score:.3f}")
    return {"neo4j": neo_score, "pdf": pdf_score}

# Example:
# compare_with_scores("Explain the UE registration flow and the NFs involved.")

In [ ]:
compare_with_scores("Enable each network function to declare the services it exposes, including version and supported protocols.")

Grounding similarity (cosine to retrieved context):
  Neo4j: 0.460
  PDF  : 0.502


{'neo4j': 0.4596777566713459, 'pdf': 0.5021152072737562}

In [ ]:
show_comparison("Enable each network function to declare the services it exposes, including version and supported protocols.")

QUESTION:
Enable each network function to declare the services it exposes, including version and supported protocols.
[Neo4j] time: 11.66s
To translate the high-level requirements for a 5G Core network into a YANG configuration, we need to
define a YANG module that allows each network function to declare the services it exposes, including
version and supported protocols. Below is a YANG configuration that adheres to the specified
requirements.  ### YANG Module  ```yang module 5g-core-services {     yang-version 1.1;
namespace "urn:5g-core-services";     prefix "5gcs";      import ietf-yang-types {         prefix
"yang";     }      organization "Your Organization";     contact "your.email@example.com";
description         "This module defines the services exposed by network functions in a 5G Core
network.";      typedef protocol-type {         type enumeration {             enum "HTTP" {
description "Hypertext Transfer Protocol";             }             enum "HTTPS" {
description "Hyp